# Рекомендация мобильного тарифа

**Описание проекта**

Оператор мобильной связи выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75.

## Откроем и изучим файл

Чтобы изучить данные таблицы, сделать необходимые рассчёты и провести исследование сразу импортируем библиотеки, который нам потребуются в данном проекте:

In [1]:
from io import BytesIO
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 

import numpy as np 
import pandas as pd
import requests

Напишем функцию, чтобы прочитать и сохранить данные из Google Sheets

In [2]:
def table(spreadsheet_id):
    spreadsheet_id = spreadsheet_id
    file_name = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_id)
    r = requests.get(file_name)
    table = pd.read_csv(BytesIO(r.content))
    return table

Прочитаем и сохраним файл, который предоставил заказчик

In [3]:
df = table('1srONTJ3arz9RzNbvnzCv9axNq5_jT10PoTflbj86PKM') # чтение файла с данными и сохранение в df 
df.head(10) # получение первых 10 строк таблицы df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


Получим общую информацию о таблице

In [4]:
df.info() # получение общей информации о данных в таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


В таблице 5 столбцов. Типы данных в столбцах — `float`, `int`.

Согласно документации к данным каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
* сalls — количество звонков,
* minutes — суммарная длительность звонков в минутах,
* messages — количество sms-сообщений,
* mb_used — израсходованный интернет-трафик в Мб,
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

**Вывод**

Нам дана таблица из 3214 строк с данными клиентов за месяц с указанием совершённых звонков, потраченных минут и интернет трафика, отправленных сообщений и названием тарифа клиента.

Предварительно можно утверждать, что, данных достаточно для проведения исследования.

В ТЗ сказано, что предобработка данных не понадобится. Это подтверждается тем, что в данных отсуствуют пропуски и нет расхождений с хорошим стилем.

## Разобьём данные на выборки

Сделаем 3 выборки - обучающую, валидационную и тестовую в пропорциях 60%, 20% и 20% соответственно.
<br>Для этого разделим весь датасет сначала в пропорциях 80% к 20%, а потом полученные 80% разделим на 75% и 25%, чтобы пропорции первого датасета были были 3:1:1.

In [5]:
train_valid, test = train_test_split(df, test_size=0.2, random_state=12345) 
train, valid = train_test_split(train_valid, test_size=0.25, random_state=12345)

**Вывод**

Мы разделили датасет на три выборки:
* обучающую, чтобы обучить модели
* валидационную, чтобы проверять модель на переобучение
* тестовую, чтобы правильно оценить готовую модель

Можно приступать к исследованиям.

## Исследуем модели

Разделим учебную, валидационные и тестовые выборки на признаки и цели:

In [6]:
features_train = train.drop(['is_ultra'], axis=1)
target_train = train['is_ultra']
features_valid = valid.drop(['is_ultra'], axis=1)
target_valid = valid['is_ultra']
features_test = test.drop(['is_ultra'], axis=1)
target_test = test['is_ultra']

### Обучающее дерево

Посмотрим на какой глубине дерева оптимально будет работать модель и будет выдавать наилучшее значение точности модели

In [7]:
best_model_tree = None
best_result_tree = 0
for i in range(1, 15):
    model_tree = DecisionTreeClassifier(max_depth = i, random_state=12345) # зададим параметры модели обучающего дерева
    model_tree.fit(features_train, target_train) # обучаем модель на учебной выборке
    predictions_valid = model_tree.predict(features_valid) # проверим предсказания модели на валидационных признаках
    result = accuracy_score(target_valid, predictions_valid)
    if result > best_result_tree:
        best_result_tree = result # сохраним наилучшее значение метрики accuracy на валидационных данных 
        best_model_tree = model_tree#  сохраним наилучшее значение метрики accuracy на валидационных данных

print("Accuracy наилучшей модели на валидационной выборке:", best_result_tree)        
print("Наилучшая модель с параметрами:", best_model_tree)  

Accuracy наилучшей модели на валидационной выборке: 0.7744945567651633
Наилучшая модель с параметрами: DecisionTreeClassifier(max_depth=7, random_state=12345)


**Вывод**

На валидационной выборке модель обучающее дерево показала лучшие результаты точности предсказания `Accuracy = 0.7744945567651633` с глубиной дерева `max_depth = 7`.

### Случайный лес

Определим с каким числом деревьев будет лучший результат точности модели и на какой глубине дерева

In [8]:
best_model_forest = None
best_result_forest = 0
for est in range(1, 100): # переберём с помощью цикла значения деревьев от 1 до 100
    for i in range(1, 15): # переберём с помощью цикла значения глубины деревьева от 1 до 15
        model_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = i) # зададим параметры модели случайный лес
        model_forest.fit(features_train, target_train) # обучим модель на обучающей выборке
        result = model_forest.score(features_valid, target_valid) # посчитаем качество модели на валидационной выборке
        if result > best_result_forest:
            best_model_forest = model_forest# сохраним наилучшую модель
            best_result_forest = result#  сохраним наилучшее значение метрики accuracy на валидационных данных

print("Accuracy наилучшей модели на валидационной выборке:", best_result_forest)        
print("Наилучшая модель с параметрами:", best_model_forest)  

Accuracy наилучшей модели на валидационной выборке: 0.80248833592535
Наилучшая модель с параметрами: RandomForestClassifier(max_depth=12, n_estimators=58, random_state=12345)


**Вывод**

На валидационной выборке модель случайный лес показала лучшие результаты точности предсказания `Accuracy = 0.80248833592535` с числом деревьев `n_estimators=58` и глубиной дерева `max_depth=12`.

### Логистическая регрессия

Определим с каким числом итераций и каким решателем будет лучший результат точности модели

In [9]:
best_model_logist = None
best_result_logist = 0
for i in range(10000, 1000000, 50000): # перебираем число итераций с шагом в 50000
    for solver in ('newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'): # перебираем варианты решателя
        model_logist = LogisticRegression(random_state=12345, solver=solver, max_iter=i) # инициализируем модель логистической регрессии с параметрами
        model_logist.fit(features_train, target_train) # обучим модель на тренировочной выборке
        result_logist = model_logist.score(features_valid, target_valid) # получим метрику качества модели на валидационной выборке
        if result_logist > best_result_logist:
            best_model_logist = model_logist# сохраним наилучшую модель
            best_result_logist = result_logist#  сохраним наилучшее значение метрики accuracy на валидационных данных

print("Accuracy наилучшей модели на валидационной выборке:", best_result_logist)        
print("Наилучшая модель с параметрами:", best_model_logist)         

D:\Anaconda\lib\site-packages\scipy\optimize\linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
D:\Anaconda\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
D:\Anaconda\lib\site-packages\scipy\optimize\linesearch.py:437: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
D:\Anaconda\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
D:\Anaconda\lib\site-packages\sklearn\utils\optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
D:\Anaconda\lib\site-packages\scipy\optimize\linesearch.py:478: LineSearchWarning: The line search algori

D:\Anaconda\lib\site-packages\scipy\optimize\linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
D:\Anaconda\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
D:\Anaconda\lib\site-packages\scipy\optimize\linesearch.py:437: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
D:\Anaconda\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
D:\Anaconda\lib\site-packages\sklearn\utils\optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
D:\Anaconda\lib\site-packages\scipy\optimize\linesearch.py:478: LineSearchWarning: The line search algori

Accuracy наилучшей модели на валидационной выборке: 0.7262830482115086
Наилучшая модель с параметрами: LogisticRegression(max_iter=10000, random_state=12345, solver='newton-cg')


**Вывод**

На валидационной выборке модель логистическая регрессия показала результат точности предсказания `Accuracy =  0.7262830482115086` с параметрами `max_iter=10000, random_state=12345, solver='newton-cg'`.

Самой точной оказалась модель случайный лес с показателем `accuracy - 0.80248833592535` и параметрами `max_depth=12, n_estimators=58, random_state=12345`.

## Проверьте модель на тестовой выборке

Протестируем модель обучающее дерево с наилучшими параметрами выбранными ранее

In [10]:
model_tree = DecisionTreeClassifier(max_depth = 7, random_state=12345) # зададим параметры модели обучающего дерева
model_tree.fit(features_train, target_train) # обучаем модель на учебной выборке
predictions_test = model_tree.predict(features_test) # проверим предсказания модели на валидационных признаках
result_tree = accuracy_score(target_test, predictions_test)
print("Accuracy:", result_tree)

Accuracy: 0.7884914463452566


Протестируем модель случайный лес с наилучшими параметрами выбранными ранее

In [11]:
model_forest = RandomForestClassifier(max_depth=12, n_estimators=58, random_state=12345) # зададим параметры модели случайный лес
model_forest.fit(features_train, target_train) # обучим модель на обучающей выборке
result_forest = model_forest.score(features_test, target_test) # посчитаем качество модели на валидационной выборке
print("Accuracy:", result_forest)

Accuracy: 0.7947122861586314


Протестируем модель логистическая регрессия

In [12]:
model_logist = LogisticRegression(max_iter=10000, random_state=12345, solver='newton-cg') # инициализируем модель логистической регрессии с параметрами
model_logist.fit(features_train, target_train) # обучим модель на тренировочной выборке
result_logist_test = model_logist.score(features_test, target_test) # получим метрику качества модели на валидационной выборке

print("Accuracy:", result_logist_test)  

Accuracy: 0.7589424572317263


D:\Anaconda\lib\site-packages\scipy\optimize\linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
D:\Anaconda\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
D:\Anaconda\lib\site-packages\scipy\optimize\linesearch.py:437: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
D:\Anaconda\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
D:\Anaconda\lib\site-packages\sklearn\utils\optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


**Вывод**

На тестовой выборке были проверены модели, которые обучали ранее - обучающее дерево, случайный лес и логистическая регрессия.

На тестовой выборке самой точной оказалась модель случайный лес с показателем `accuracy - 0.7947122861586314`.

При этом на тестовой выборке точность моделей обучающее дерево и логистическая регрессия улучшили свою точность по  сравнению с валидационной выборкой, но до случайного леса по точности не дошли, хотя обучающее дерево достаточно близко.

## (бонус) Проверьте модели на адекватность

Построим простую модель с помощью `DummyClassifier` и попробуем 3 стратегии, чтобы убедиться, что наша модель лучше

In [13]:
for strategy in ('most_frequent', 'stratified', 'uniform'): # переберём варианты стратегий
    model_dummy = DummyClassifier(strategy=strategy) # инициализируем модель 
    model_dummy.fit(features_train, target_train) # обучим модель на тренировочной выборке
    result_dummy = model_dummy.score(features_test, target_test)
    print(f'Accuracy: {result_dummy} - Модель: {model_dummy}') 

Accuracy: 0.6951788491446346 - Модель: DummyClassifier(strategy='most_frequent')
Accuracy: 0.5816485225505443 - Модель: DummyClassifier(strategy='stratified')
Accuracy: 0.5178849144634525 - Модель: DummyClassifier(strategy='uniform')


**Вывод**

Максимальное значение точности модели построенной с помощью DummyClassifier равно `accuracy - 0.6951788491446346`.

При этом выбранная нами модель случайный лес на тестовой выборке дала показатель точности `accuracy - 0.7947122861586314`.

На основании этого можно предположить, что все наши модели вменяемы и адекватны, тк у всех показатель точности выше, но использовать разумеется лучше самую точную.

## Общий вывод

Мы построили несколько моделей для задачи классификации, которые выберут подходящий тариф. 

В ТЗ требование к точности составляло 0.75.

На тестовой выборке были проверены модели, которые обучали ранее - обучающее дерево, случайный лес и логистическая регрессия.

У них у всех точность был выше 0.75.

На тестовой выборке самой точной оказалась модель случайный лес с показателем `accuracy - 0.7947122861586314`.

При этом на валидационной выборке модель логистической регрессии выдавала точность менее 0.75, следовательно её не стоит исспользовать для классификации.

Для классификации можно взять модели обучающее дерево и случайный лес, но самая высокая точность на валидационной и тестовой выборке была у модели случайный лес.
